# Mirascope Quickstart Guide

1. [Setup](#Setup)
2. [Prompt Templates](#Prompt-Templates)
3. [Basic LLM Call](#Basic-LLM-Call)
4. [Streaming Responses](#Streaming-Responses)
5. [Response Models](#Response-Models)
6. [Asynchronous Processing](#Asynchronous-Processing)
7. [JSON Mode](#JSON-Mode)
8. [Output Parsers](#Output-Parsers)

## Setup

Mirascope supports various LLM providers, including [OpenAI](https://openai.com/), [Anthropic](https://www.anthropic.com/), [Mistral](https://mistral.ai/), [Gemini](https://gemini.google.com), [Groq](https://groq.com/), [Cohere](https://cohere.com/), [LiteLLM](https://www.litellm.ai/), [Azure AI](https://azure.microsoft.com/en-us/solutions/ai), and [Vertex AI](https://cloud.google.com/vertex-ai). For the purposes of this guide, we will be using OpenAI. Let's start by installing Mirascope and its dependencies:

In [25]:
!pip install "mirascope[openai]"

  Using cached filelock-3.15.4-py3-none-any.whl.metadata (2.9 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 891.9/891.9 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 10.4 MB/s eta 0:00:00a 0:00:01
Using cached filelock-3.15.4-py3-none-any.whl (16 kB)


This command installs Mirascope along with the necessary packages for the OpenAI integration.

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"
# Set the appropriate API key for the provider you're using

## Prompt Templates

Prompt templates in Mirascope allow you to create dynamic and reusable prompts.

In [ ]:
from mirascope.core import prompt_template


@prompt_template("What is the capital of {country}?")
def get_capital_prompt(country: str): ...


print(get_capital_prompt("Japan"))

In this example:
1. The `@prompt_template` decorator defines the structure of the prompt.
2. `{country}` is a placeholder that gets replaced with the matching argument passed to the function.

Here's an example of a multi-line prompt:

In [ ]:
@prompt_template(
    """
    SYSTEM:
    You are a knowledgeable assistant specializing in world geography.
    Your task is to provide detailed information about the capital cities of various countries.
    
    USER:
    Please provide the following information about the capital of {country}:
    1. The name of the capital city
    2. Its population (approximate is fine)
    3. Three major landmarks or attractions
    """
)
def get_capital_info(country: str): ...


capital_info = get_capital_info("France")
print(capital_info.content)

This example demonstrates:
1. A multi-line prompt template with proper indentation.
2. Use of "SYSTEM" and "USER" keywords to define messages with different roles.
3. A complex, multi-part query within the USER message.
4. Use of a variable (`{country}`) to create a dynamic prompt.

Prompt templates like this improve code readability, maintain consistency in prompts, and make it easier to experiment with and modify prompts for effective prompt engineering. They are particularly useful for complex queries that require detailed instructions or context.

For more detailed information on prompt templates and advanced usage, check out our [documentation on Prompts](https://docs.mirascope.io/learn/prompts).

## Basic LLM Call

The `call` decorator transforms functions with prompt templates into LLM API calls.

In [ ]:
from mirascope.core import openai

# import provider-specific modules to use those providers (e.g. anthropic, gemini, etc.)


@openai.call("gpt-4o-mini")
@prompt_template("What is the capital of {country}?")
def get_capital(country: str): ...


response = get_capital("Japan")
print(response.content)

For more advanced usage of LLM calls, refer to our [documentation on Calls](https://docs.mirascope.io/learn/calls).

## Streaming Responses

Streaming allows you to process LLM responses in real-time.

In [ ]:
@openai.call("gpt-4o-mini", stream=True)
@prompt_template("Provide a brief description of {city}.")
def stream_city_info(city: str): ...


for chunk, _ in stream_city_info("Tokyo"):
    print(chunk.content, end="", flush=True)

For more information on streaming, including advanced techniques for processing streamed content, see our [documentation on Streams](https://docs.mirascope.io/learn/streams).

## Response Models

Response models allow you to structure and validate the output from LLMs.

In [23]:
from pydantic import BaseModel


class Capital(BaseModel):
    city: str
    country: str


@openai.call("gpt-4o-mini", response_model=Capital)
@prompt_template("{query}")
def extract_capital(query: str): ...


capital = extract_capital("The capital of France is Paris")
print(capital)

city='Paris' country='France'


For more details on response models, including advanced validation techniques, check our [documentation on Response Models](https://docs.mirascope.io/learn/response-models).

## Asynchronous Processing

Mirascope supports asynchronous processing for efficient parallel execution of multiple LLM calls.

In [22]:
import asyncio


@openai.call("gpt-4o-mini", response_model=Capital)
@prompt_template("What is the capital of {country}?")
async def get_capital_async(country: str): ...


async def main():
    countries = ["France", "Japan", "Brazil"]
    tasks = [get_capital_async(country) for country in countries]
    capitals = await asyncio.gather(*tasks)
    for capital in capitals:
        print(f"The capital of {capital.country} is {capital.city}")


# await main() when running in a Jupyter notebook
await main()

# asyncio.run(main()) when running in a Python script
# asyncio.run(main())

The capital of France is Paris
The capital of Japan is Tokyo
The capital of Brazil is Brasilia


For more advanced usage of asynchronous processing, including async streaming, see our [documentation on Async](https://docs.mirascope.io/learn/async).

## JSON Mode

JSON mode allows you to directly parse LLM outputs as JSON.

In [ ]:
@openai.call("gpt-4o-mini", json_mode=True)
@prompt_template("Provide information about {city} in JSON format")
def city_info(city: str): ...


response = city_info("Tokyo")
print(response.content)  # This will be a JSON-formatted string

Note that not all providers have an explicit JSON mode. For those providers, we attempt to instruct the model to provide JSON; however, there is no guarantee that it will output only JSON (it may start with some text like "Here is the JSON: ..."). This is where [Output Parsers](https://docs.mirascope.io/learn/output-parsers) can be useful.

For more information on JSON mode, refer to our [documentation on JSON Mode](https://docs.mirascope.io/learn/json-mode).

## Output Parsers

Output parsers allow you to process LLM responses in custom formats. They are particularly useful when working with JSON outputs, especially for providers like Anthropic that don't have a strict JSON mode.

In [28]:
!pip install "mirascope[anthropic]"

In [30]:
import json

from mirascope.core import anthropic


def only_json(response: anthropic.AnthropicCallResponse) -> str:
    json_start = response.content.index("{")
    json_end = response.content.rfind("}")
    return response.content[json_start : json_end + 1]


@anthropic.call("claude-3-5-sonnet-20240620", json_mode=True, output_parser=only_json)
@prompt_template("Extract {fields} from the following text: {text}")
def json_extraction(text: str, fields: list[str]): ...


json_response = json_extraction(
    text="The capital of France is Paris",
    fields=["capital", "country"],
)
print(json.loads(json_response))

{'capital': 'Paris', 'country': 'France'}


For more information on output parsers, see our [documentation on Output Parsers](https://docs.mirascope.io/learn/output-parsers).

This concludes our Quickstart Guide to Mirascope. We've covered the main features of the library, including prompt templates, basic calls, streaming, response models, asynchronous processing, JSON mode, and output parsers. Each of these features can be combined and customized to create powerful, flexible AI applications.

For more detailed information on each of these topics and advanced usage, including dynamic configuration and chaining, please refer to our comprehensive [Learn documentation](https://docs.mirascope.io/learn).